In [1]:
%run init.ipynb

matchzoo version 2.2.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.visible_device_list="1"
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
def load_filtered_data(preprocessor, data_type):
    assert ( data_type in ['train', 'dev', 'test'])
    data_pack = mz.datasets.wiki_qa.load_data(data_type, task='ranking')

    if data_type == 'train':
        X, Y = preprocessor.fit_transform(data_pack).unpack()
    else:
        X, Y = preprocessor.transform(data_pack).unpack()

    new_idx = []
    for i in range(Y.shape[0]):
        if X["length_left"][i] == 0 or X["length_right"][i] == 0:
            continue
        new_idx.append(i)
    new_idx = np.array(new_idx)
    print("Removed empty data. Found ", (Y.shape[0] - new_idx.shape[0]))

    for k in X.keys():
        X[k] = X[k][new_idx]
    Y = Y[new_idx]

    pos_idx = (Y == 1)[:, 0]
    pos_qid = X["id_left"][pos_idx]
    keep_idx_bool = np.array([ qid in pos_qid for qid in X["id_left"]])
    keep_idx = np.arange(keep_idx_bool.shape[0])
    keep_idx = keep_idx[keep_idx_bool]
    print("Removed questions with no pos label. Found ", (keep_idx_bool == 0).sum())

    print("shuffling...")
    np.random.shuffle(keep_idx)
    for k in X.keys():
        X[k] = X[k][keep_idx]
    Y = Y[keep_idx]

    return X, Y, preprocessor

In [4]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=20,
                                                  fixed_length_right=40,
                                                  remove_stop_words=False)
train_X, train_Y, preprocessor = load_filtered_data(preprocessor, 'train')
val_X, val_Y, _ = load_filtered_data(preprocessor, 'dev')
pred_X, pred_Y, _ = load_filtered_data(preprocessor, 'test')

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 13071.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 8909.52it/s]
Building Vocabulary from a datapack.: 100%|██████████| 473730/473730 [00:00<00:00, 5781480.44it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 14829.27it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 8690.03it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 149305.25it/s]
Removed empty data. Found  26
Removed questions with no pos label. Found  11675
shuffling...
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 296/296 [00:00<00:00, 15745.06it/s]
Processing text_righ

In [5]:
from tensorflow.keras.optimizers import Adam
import matchzoo

model = matchzoo.contrib.models.ESIM()

# update `input_shapes` and `embedding_input_dim`
# model.params['task'] = mz.tasks.Ranking() 
# or 
model.params['task'] = mz.tasks.Ranking()
model.params.update(preprocessor.context)

model.params['mask_value'] = 0
model.params['lstm_dim'] = 300
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = False
model.params['dropout_rate'] = 0.5

model.params['mlp_num_units'] = 300
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = Adam(lr=1e-4)
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

/home/songzy/.virtualenvs/matchzoo/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          [(None, 20)]         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5244600     text_left[0][0]                  
                                                                 text_right[0][

In [6]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'], initializer=lambda: 0)
model.load_embedding_matrix(embedding_matrix)

In [7]:
# train as ranking task
model.params['task'] = mz.tasks.Ranking()
evaluate = mz.callbacks.EvaluateAllMetrics(model,
                                           x=pred_X,
                                           y=pred_Y,
                                           once_every=1,
                                           batch_size=len(pred_Y))
history = model.fit(x = [train_X['text_left'],
                         train_X['text_right']],                  # (20360, 1000)
                    y = train_Y,                                  # (20360, 2)
                    validation_data = (val_X, val_Y),
                    callbacks=[evaluate],
                    batch_size = 32,
                    epochs = 10)

Epoch 1/10
271/271 [==============================] - ETA: 0s - loss: 0.1077/home/songzy/.virtualenvs/matchzoo/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:588: UserWarning: Input dict contained keys ['id_left', 'length_left', 'id_right', 'length_right'] which did not match any model input. They will be ignored by the model.
  warnings.warn(
271/271 [==============================] - 78s 265ms/step - loss: 0.1077 - val_loss: 0.1052
Validation: mean_average_precision(0.0): 0.6259904138399054
Epoch 2/10
271/271 [==============================] - 70s 257ms/step - loss: 0.0991 - val_loss: 0.0959
Validation: mean_average_precision(0.0): 0.6464170163398558
Epoch 3/10
271/271 [==============================] - 67s 246ms/step - loss: 0.0947 - val_loss: 0.0917
Validation: mean_average_precision(0.0): 0.6386691486046969
Epoch 4/10
271/271 [==============================] - 66s 244ms/step - loss: 0.0916 - val_loss: 0.0962
Validation: mean_average_precision(0.0): 0.6457

In [8]:
# train as classification task 

from tensorflow.keras.utils import to_categorical
train_Y = to_categorical(train_Y)
val_Y = to_categorical(val_Y)

model.params['task'] = mz.tasks.Classification(num_classes=2)

history = model.fit(x = [train_X['text_left'],
                         train_X['text_right']],                  # (20360, 1000)
                    y = train_Y,                                  # (20360, 2)
                    validation_data = (val_X, val_Y),
                    batch_size = 32,
                    epochs = 10)

Epoch 1/10
271/271 [==============================] - 71s 251ms/step - loss: 0.2572 - val_loss: 0.2506
Epoch 2/10
271/271 [==============================] - 67s 247ms/step - loss: 0.2514 - val_loss: 0.2503
Epoch 3/10
271/271 [==============================] - 66s 242ms/step - loss: 0.2512 - val_loss: 0.2503
Epoch 4/10
271/271 [==============================] - 66s 242ms/step - loss: 0.2511 - val_loss: 0.2502
Epoch 5/10
271/271 [==============================] - 68s 250ms/step - loss: 0.2510 - val_loss: 0.2504
Epoch 6/10
271/271 [==============================] - 68s 249ms/step - loss: 0.2510 - val_loss: 0.2502
Epoch 7/10
271/271 [==============================] - 68s 250ms/step - loss: 0.2510 - val_loss: 0.2502
Epoch 8/10
271/271 [==============================] - 67s 245ms/step - loss: 0.2509 - val_loss: 0.2500
Epoch 9/10
271/271 [==============================] - 68s 250ms/step - loss: 0.2509 - val_loss: 0.2501
Epoch 10/10
271/271 [==============================] - 66s 244ms/step - l